In [11]:
#CIFAR-10 影像識別

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [2]:
train_split, valid_split = ['train[:90%]', 'train[90%:]']
train_data, info = tfds.load('cifar10', split=train_split, with_info=True)
valid_data = tfds.load('cifar10', split = valid_split)
test_data = tfds.load('cifar10', split = 'test')
print(info)

tfds.core.DatasetInfo(
    name='cifar10',
    full_name='cifar10/3.0.2',
    description="""
    The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_path='/Users/kai/tensorflow_datasets/cifar10/3.0.2',
    download_size=162.17 MiB,
    dataset_size=132.40 MiB,
    features=FeaturesDict({
        'id': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=50000, num_shards=1>,
    },
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multiple

In [4]:
labels_dict = dict(enumerate(info.features['label'].names))
labels_dict

{0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}

In [5]:
train_dict = {}
for data in train_data:
    label = data['label'].numpy()
    train_dict[label] = train_dict.setdefault(label, 0) + 1
print(train_dict)

{7: 4500, 8: 4456, 4: 4497, 6: 4505, 5: 4503, 2: 4522, 9: 4453, 3: 4526, 0: 4530, 1: 4508}


In [6]:
output = np.zeros((32 * 8, 32 * 8, 3), dtype = np.uint8)
row = 0
for data in train_data.batch(8).take(8):
    output[:, row*32:(row+1)*32] = np.vstack(data['image'].numpy())
    row += 1
plt.figure(figsize = (8, 8))
plt.imshow(output)

In [7]:
def parse_fn(dataset):
    x = tf.cast(dataset['image'], tf.float32) / 255.
    y = tf.one_hot(dataset['label'], 10)
    return x, y

In [8]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 64
train_num = int(info.splits['train'].num_examples / 10) * 9

In [9]:
train_data = train_data.shuffle(train_num)
train_data = train_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

valid_data = valid_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [12]:
#訓練方式：全神經網路
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Flatten()(inputs)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model_1 = keras.Model(inputs, outputs, name='model-1')
model_1.summary()

Model: "model-1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               393344    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               1315

In [13]:
model_dir = 'lab4-logs/models'
os.makedirs(model_dir)

In [14]:
log_dir = os.path.join('lab4-logs', 'model-1')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(model_dir + '/Best-model-1.h5',
                                            monitor='val_categorical_accuracy',
                                            save_best_only=True,
                                            mode='max')
model_1.compile(keras.optimizers.Adam(),
               loss=keras.losses.CategoricalCrossentropy(),
               metrics=[keras.metrics.CategoricalAccuracy()])

In [15]:
history_1 = model_1.fit(train_data,
                       epochs=100,
                       validation_data=valid_data,
                       callbacks=[model_cbk, model_mckp])

Epoch 1/100
704/704 [==============================] - 9s 11ms/step - loss: 2.2137 - categorical_accuracy: 0.1416 - val_loss: 2.1401 - val_categorical_accuracy: 0.1766
Epoch 2/100
704/704 [==============================] - 7s 10ms/step - loss: 2.1364 - categorical_accuracy: 0.1665 - val_loss: 2.0796 - val_categorical_accuracy: 0.1854
Epoch 3/100
704/704 [==============================] - 7s 10ms/step - loss: 2.1198 - categorical_accuracy: 0.1746 - val_loss: 2.0778 - val_categorical_accuracy: 0.1982
Epoch 4/100
704/704 [==============================] - 7s 10ms/step - loss: 2.1123 - categorical_accuracy: 0.1788 - val_loss: 2.0719 - val_categorical_accuracy: 0.1724
Epoch 5/100
704/704 [==============================] - 7s 10ms/step - loss: 2.0977 - categorical_accuracy: 0.1898 - val_loss: 2.0482 - val_categorical_accuracy: 0.2166
Epoch 6/100
704/704 [==============================] - 7s 10ms/step - loss: 2.0943 - categorical_accuracy: 0.1924 - val_loss: 2.0207 - val_categorical_accuracy:

704/704 [==============================] - 7s 10ms/step - loss: 2.0191 - categorical_accuracy: 0.2181 - val_loss: 1.9823 - val_categorical_accuracy: 0.2552
Epoch 50/100
704/704 [==============================] - 7s 10ms/step - loss: 2.0147 - categorical_accuracy: 0.2197 - val_loss: 2.0419 - val_categorical_accuracy: 0.2308
Epoch 51/100
704/704 [==============================] - 7s 10ms/step - loss: 2.0228 - categorical_accuracy: 0.2189 - val_loss: 1.9780 - val_categorical_accuracy: 0.2474
Epoch 52/100
704/704 [==============================] - 7s 10ms/step - loss: 2.0163 - categorical_accuracy: 0.2218 - val_loss: 1.9756 - val_categorical_accuracy: 0.2560
Epoch 53/100
704/704 [==============================] - 7s 10ms/step - loss: 2.0195 - categorical_accuracy: 0.2210 - val_loss: 2.0441 - val_categorical_accuracy: 0.2162
Epoch 54/100
704/704 [==============================] - 7s 10ms/step - loss: 2.0199 - categorical_accuracy: 0.2218 - val_loss: 1.9977 - val_categorical_accuracy: 0.2400

704/704 [==============================] - 7s 10ms/step - loss: 2.0157 - categorical_accuracy: 0.2256 - val_loss: 2.0083 - val_categorical_accuracy: 0.2326
Epoch 98/100
704/704 [==============================] - 7s 10ms/step - loss: 2.0251 - categorical_accuracy: 0.2244 - val_loss: 2.0570 - val_categorical_accuracy: 0.2022
Epoch 99/100
704/704 [==============================] - 7s 10ms/step - loss: 2.0191 - categorical_accuracy: 0.2263 - val_loss: 2.0137 - val_categorical_accuracy: 0.2406
Epoch 100/100
704/704 [==============================] - 7s 10ms/step - loss: 2.0165 - categorical_accuracy: 0.2252 - val_loss: 2.0033 - val_categorical_accuracy: 0.2374


In [16]:
#訓練方式：卷積神經網路

inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.Conv2D(256, (3, 3), activation='relu')(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model_2 = keras.Model(inputs, outputs, name='model-2')
model_2.summary()


log_dir = os.path.join('lab4-logs', 'model-2')
model_cbk = keras.callbacks.TensorBoard(log_dir = log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(model_dir + '/Best-model-2.h5',
                                            monitor='val_categorical_accuracy',
                                            save_best_only=True,
                                            mode='max')
model_2.compile(keras.optimizers.Adam(),
               loss = keras.losses.CategoricalCrossentropy(),
               metrics=[keras.metrics.CategoricalAccuracy()])

history_2 = model_2.fit(train_data,
                       epochs=100,
                       validation_data=valid_data,
                       callbacks=[model_cbk, model_mckp])

Model: "model-2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         295040    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)          7379

704/704 [==============================] - 23s 33ms/step - loss: 0.1967 - categorical_accuracy: 0.9338 - val_loss: 1.6770 - val_categorical_accuracy: 0.7114
Epoch 39/100
704/704 [==============================] - 23s 33ms/step - loss: 0.2035 - categorical_accuracy: 0.9312 - val_loss: 1.5166 - val_categorical_accuracy: 0.7256
Epoch 40/100
704/704 [==============================] - 23s 33ms/step - loss: 0.1858 - categorical_accuracy: 0.9379 - val_loss: 1.6578 - val_categorical_accuracy: 0.7282
Epoch 41/100
704/704 [==============================] - 23s 32ms/step - loss: 0.1825 - categorical_accuracy: 0.9395 - val_loss: 1.6925 - val_categorical_accuracy: 0.7270
Epoch 42/100
704/704 [==============================] - 23s 33ms/step - loss: 0.1758 - categorical_accuracy: 0.9421 - val_loss: 1.6784 - val_categorical_accuracy: 0.7306
Epoch 43/100
704/704 [==============================] - 23s 33ms/step - loss: 0.1846 - categorical_accuracy: 0.9398 - val_loss: 1.6742 - val_categorical_accuracy: 

704/704 [==============================] - 23s 33ms/step - loss: 0.1049 - categorical_accuracy: 0.9678 - val_loss: 2.3052 - val_categorical_accuracy: 0.7248
Epoch 87/100
704/704 [==============================] - 23s 32ms/step - loss: 0.1058 - categorical_accuracy: 0.9682 - val_loss: 2.3403 - val_categorical_accuracy: 0.7230
Epoch 88/100
704/704 [==============================] - 23s 33ms/step - loss: 0.1083 - categorical_accuracy: 0.9670 - val_loss: 2.2419 - val_categorical_accuracy: 0.7190
Epoch 89/100
704/704 [==============================] - 23s 32ms/step - loss: 0.0937 - categorical_accuracy: 0.9708 - val_loss: 2.3581 - val_categorical_accuracy: 0.7248
Epoch 90/100
704/704 [==============================] - 23s 32ms/step - loss: 0.1006 - categorical_accuracy: 0.9691 - val_loss: 2.2713 - val_categorical_accuracy: 0.7274
Epoch 91/100
704/704 [==============================] - 23s 32ms/step - loss: 0.1033 - categorical_accuracy: 0.9692 - val_loss: 2.3995 - val_categorical_accuracy: 

In [17]:
x = 3
y = 7
image_test = output[y*32:(y+1)*32, x*32:(x+1)*32, :]
plt.imshow(image_test)

In [18]:
def flip(x):
    '''
    flip image(翻轉影像)
    '''
    x = tf.image.random_flip_left_right(x)
    return x

image_2 = flip(image_test)
image = np.hstack((image_test, image_2))
plt.imshow(image)

In [19]:
def color(x):
    '''
    Color change(改變顏色)
    '''
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

In [20]:
image_2 = color(image_test)
image = np.hstack((image_test, image_2))
plt.imshow(image)

In [21]:
def rotate(x):
    '''
    Rotation image(影像旋轉)
    '''
    x = tf.image.rot90(x,tf.random.uniform(shape=[],minval=1,maxval=4,
                                          dtype=tf.int32))
    return x

image_2 = rotate(image_test)
image = np.hstack((image_test, image_2))
plt.imshow(image)

In [22]:
def zoom(x, scale_min=0.6, scale_max=1.4):
    '''
    Zoom Image(影像縮放)
    '''
    h, w, c = x.shape
    scale = tf.random.uniform([], scale_min, scale_max)
    sh = h * scale
    sw = w * scale
    x = tf.image.resize(x, (sh, sw))
    x = tf.image.resize_with_crop_or_pad(x, h, w)
    return x

In [23]:
image_2 = zoom(image_test)
image_2 = tf.cast(image_2, dtype=tf.uint8)
image = np.hstack((image_test, image_2))
plt.imshow(image)

In [24]:
train_data = tfds.load('cifar10', split=train_split)

In [25]:
def parse_aug_fn(dataset):
    '''
    Image Augmentation(影像增強) function
    '''
    x = tf.cast(dataset['image'], tf.float32) / 255.
    x = flip(x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.5, lambda: color(x), lambda: x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.75, lambda: rotate(x), lambda: x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.5, lambda: zoom(x), lambda: x)
    y = tf.one_hot(dataset['label'], 10)
    return x, y

In [26]:
train_data = train_data.shuffle(train_num)
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [27]:
for images, labels in train_data.take(1):
    images = images.numpy()
    
output = np.zeros((32 * 8, 32 * 8, 3))
for i in range(8):
    for j in range(8):
        output[i*32:(i+1)*32, j*32:(j+1)*32, :] = images[i*8+j]
plt.figure(figsize=(8, 8))
plt.imshow(output)

In [29]:
#訓練方式：使用影像增強方法訓練卷積神經網路
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.Conv2D(256, (3, 3), activation='relu')(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation = 'softmax')(x)

model_3 = keras.Model(inputs, outputs, name = 'model-3')
model_3.summary()



lod_dir = os.path.join('lab4-logs','model-3')
model_cbk = keras.callbacks.TensorBoard(log_dir = log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(model_dir + '/Best=model-3.h5',
                                            monitor='val_categorical_accuracy',
                                            save_best_only=True,
                                            mode='max')

model_3.compile(keras.optimizers.Adam(),
               loss=keras.losses.CategoricalCrossentropy(),
               metrics=[keras.metrics.CategoricalAccuracy()])


hostory_3 = model_3.fit(train_data,
                       epochs=100,
                       validation_data=valid_data,
                       callbacks=[model_cbk, model_mckp])

Model: "model-3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 256)       295168    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 9, 9, 128)         295040    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          7379

704/704 [==============================] - 24s 35ms/step - loss: 0.9759 - categorical_accuracy: 0.6698 - val_loss: 0.7787 - val_categorical_accuracy: 0.7406
Epoch 39/100
704/704 [==============================] - 24s 33ms/step - loss: 0.9706 - categorical_accuracy: 0.6733 - val_loss: 0.7583 - val_categorical_accuracy: 0.7428
Epoch 40/100
704/704 [==============================] - 24s 34ms/step - loss: 0.9614 - categorical_accuracy: 0.6748 - val_loss: 0.7132 - val_categorical_accuracy: 0.7600
Epoch 41/100
704/704 [==============================] - 23s 33ms/step - loss: 0.9568 - categorical_accuracy: 0.6773 - val_loss: 0.7451 - val_categorical_accuracy: 0.7510
Epoch 42/100
704/704 [==============================] - 24s 33ms/step - loss: 0.9492 - categorical_accuracy: 0.6803 - val_loss: 0.7291 - val_categorical_accuracy: 0.7536
Epoch 43/100
704/704 [==============================] - 23s 33ms/step - loss: 0.9544 - categorical_accuracy: 0.6774 - val_loss: 0.7279 - val_categorical_accuracy: 

704/704 [==============================] - 23s 33ms/step - loss: 0.7987 - categorical_accuracy: 0.7332 - val_loss: 0.6452 - val_categorical_accuracy: 0.7932
Epoch 87/100
704/704 [==============================] - 24s 33ms/step - loss: 0.7957 - categorical_accuracy: 0.7342 - val_loss: 0.6594 - val_categorical_accuracy: 0.7886
Epoch 88/100
704/704 [==============================] - 24s 34ms/step - loss: 0.8067 - categorical_accuracy: 0.7306 - val_loss: 0.6589 - val_categorical_accuracy: 0.7900
Epoch 89/100
704/704 [==============================] - 24s 34ms/step - loss: 0.7930 - categorical_accuracy: 0.7375 - val_loss: 0.6907 - val_categorical_accuracy: 0.7860
Epoch 90/100
704/704 [==============================] - 24s 33ms/step - loss: 0.7952 - categorical_accuracy: 0.7370 - val_loss: 0.6585 - val_categorical_accuracy: 0.7882
Epoch 91/100
704/704 [==============================] - 23s 33ms/step - loss: 0.8024 - categorical_accuracy: 0.7338 - val_loss: 0.6564 - val_categorical_accuracy: 